# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Delve into the world of artificial intelligence and unlock its full potential with DeepLearning.AI. Founded by renowned AI expert Andrew Ng, this platform offers top-notch courses on deep learning, allowing both beginners and professionals to master this cutting-edge technology. Whether you're looking to upskill in AI or dive into a new career path, DeepLearning.AI provides the tools and knowledge you need to succeed. Join the ranks of AI enthusiasts worldwide and embark on an exciting journey of learning and innovation with DeepLearning.AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Delve into the world of artificial intelligence and unlock its full potential with DeepLearning.AI. Founded by renowned AI expert Andrew Ng, this platform offers top-notch courses on deep learning, allowing both beginners and professionals to master this cutting-edge technology. Whether you're looking to upskill in AI or dive into a new career path, DeepLearning.AI provides the tools and knowledge you need to succeed. Join the ranks of AI enthusiasts worldwide and embark on an exciting journey of learning and innovation with DeepLearning.AI.

--------------------------------------------------------------------------------
Critic (to Writer):

Your blog post effecti

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'feedback': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'feedback': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'feedback': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Delve into the world of artificial intelligence and unlock its full potential with DeepLearning.AI. Founded by renowned AI expert Andrew Ng, this platform offers top-notch courses on deep learning, allowing both beginners and professionals to master this cutting-edge technology. Whether you're looking to upskill in AI or dive into a new career path, DeepLearning.AI provides the tools and knowledge you need to succeed. Join the ranks of AI enthusiasts worldwide and embark on an exciting journey of learning and innovation with DeepLearning.AI.

--------------------------------------------------------------------------------

******************************************


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

**Aggregated Feedback:**
1. **SEO Specialist (Reviewers 1 & 2):**  
   - Incorporate keywords related to deep learning, artificial intelligence, online courses, Andrew Ng, and upskilling for improved search engine visibility.
   - Utilize structured data markup like schema.org tags to give context to search engines and increase chances of getting featured snippets.
   - Enhance call-to-action with actionable language such as "Enroll Now" or "Start Learning" to boost user engagement.

2. **Ethics Reviewer:**  
   - Back any claims about the effectiveness of DeepLearning.AI courses with evidence to avoid misleading users.
   - Disclose any potential conflicts of interest to maintain transparency.
   - Obtain permission to use images, testimonials, and endorsements to avoid copyright infringement and ensure consent.

**Final Suggestions:**
- This content can benefit from incorpora

## Get the summary

In [15]:
print(res.summary)

Title: Master Deep Learning and AI with DeepLearning.AI: A Comprehensive Review

Discover the vast potential of artificial intelligence through DeepLearning.AI, a leading online platform founded by AI luminary Andrew Ng. Offering a wide range of courses, from beginner to advanced levels, DeepLearning.AI equips learners with the essential skills needed to navigate the dynamic field of deep learning. Gain expertise, upskill in AI, and explore new career avenues with confidence. Enhance your knowledge, propel your career, and unleash your creativity with DeepLearning.AI. Enroll now and embark on a transformative learning journey!


In [18]:
from pprint import pprint
pprint(res.chat_history)

[{'content': '\n'
             '        Write a concise but engaging blogpost about\n'
             '       DeepLearning.AI. Make sure the blogpost is\n'
             '       within 100 words.\n'
             '       ',
  'role': 'assistant'},
 {'content': 'Title: Unleashing the Power of Deep Learning with '
             'DeepLearning.AI\n'
             '\n'
             'Delve into the world of artificial intelligence and unlock its '
             'full potential with DeepLearning.AI. Founded by renowned AI '
             'expert Andrew Ng, this platform offers top-notch courses on deep '
             'learning, allowing both beginners and professionals to master '
             "this cutting-edge technology. Whether you're looking to upskill "
             'in AI or dive into a new career path, DeepLearning.AI provides '
             'the tools and knowledge you need to succeed. Join the ranks of '
             'AI enthusiasts worldwide and embark on an exciting journey of '
         